# Imports

In [3]:
import pandas as pd # to interact with dataframes 
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder #for preprocessing
import joblib #to save encoders and models 
import os #to interact with hardware and create folders 
import warnings
warnings.simplefilter(action='ignore')

# Constants

In [4]:


# Initialize filepaths 

CLEAN_SQL_PATH = 'sql_files/clean/'
PREPROCESSED_FILE_PATH = 'preprocessed/'



# Reading the Data

In [5]:
data = pd.read_csv(os.path.join(PREPROCESSED_FILE_PATH, '_All_States_Cleaned.csv')) # read the cleaned data 


In [6]:
data.head() # see first 5 rows of the data 

,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,state,amm_0,amm_1,...,Yucaipa,Yucca Valley,Yukon,Yulee,Yuma,Zachary,Zanesville,Zebulon,Zephyrhills,Zion
0,1,1.0,1.0,1,1,2195.0,542,CA,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1.5,3.0,1,3,1250.0,1500,VA,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2.0,3.0,1,0,1395.0,1650,NC,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1.0,2.0,1,3,1600.0,820,CA,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1.0,1.0,1,3,975.0,624,NM,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.value_counts('state')[:10]

state
TX    10021
CA     9797
VA     7316
NC     5799
CO     5517
FL     5252
MA     4856
MD     4553
GA     4367
NJ     4172
dtype: int64

In [8]:
# create a copy of the subsets of the dataframes for each state.

In [9]:
tx = data[data['state'] == 'TX'].copy()
ca = data[data['state'] == 'CA'].copy()
va = data[data['state'] == 'VA'].copy()
nc = data[data['state'] == 'NC'].copy()
co = data[data['state'] == 'CO'].copy()
fl = data[data['state'] == 'FL'].copy()
ma = data[data['state'] == 'MA'].copy()
md = data[data['state'] == 'MD'].copy()

In [10]:
df_list_state = [tx, ca, va, nc, co, fl, ma, md] #store the dataframes in a list
states = ['TX', 'CA', 'VA', 'NC', 'CO', 'FL', 'MA', 'MD'] # store the state names in a list
df_dict_state = dict(zip(states, df_list_state)) #create a dictionary of dataframes with states as key

In [11]:
data.head()

,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,state,amm_0,amm_1,...,Yucaipa,Yucca Valley,Yukon,Yulee,Yuma,Zachary,Zanesville,Zebulon,Zephyrhills,Zion
0,1,1.0,1.0,1,1,2195.0,542,CA,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1.5,3.0,1,3,1250.0,1500,VA,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2.0,3.0,1,0,1395.0,1650,NC,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1.0,2.0,1,3,1600.0,820,CA,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1.0,1.0,1,3,975.0,624,NM,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
def preprocess(df_dict):
    '''
    Cleans and preprocesses the data for future use 
    Cleaning:
        - Removes duplicate rows (if they exist)
        - Removes the state column as the dataframes are already separated 
        - Removes outliers on the basis of price column using lower limit as (Q1-1.5*IQR) and 
        upper limit as (Q3+1.5*IQR)
        
    Args:
        df_dict - the dictionary containing states as keys and the dataframe corresponding to them as values 
    Returns:
        None 
        
    '''
    
    for state, df in df_dict.items():
            
        # Cleaning the data 
        # Dropping the city column
        df.drop(['state'], axis=1, inplace=True)
        # Dropping duplicate rows (if they exist)
        df.drop_duplicates(inplace=True)
        # Removing outliers : price
        desc = df['price'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['price']>=lower_lim)&(df['price']<=upper_lim)]
        # Removing outliers : square_feet
        desc = df['square_feet'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['square_feet']>=lower_lim)&(df['square_feet']<=upper_lim)]
        
        # one hot encoding cityname
        #dummies = pd.get_dummies(df.cityname)
        #df = pd.concat([df,dummies], axis = 'columns')
        #df = df.drop('cityname',axis=1)
            
        # since the data is cleaned now, we can store it in clean_dict 
        clean_df = df.copy()
        # Saving the dataframes 
        if not os.path.exists(PREPROCESSED_FILE_PATH):
                os.makedirs(PREPROCESSED_FILE_PATH)
        df.to_csv(os.path.join(PREPROCESSED_FILE_PATH, f'{state}.csv'),index=False)            

In [13]:
preprocess(df_dict_state)

In [14]:
# Fetch the data  
data1 = pd.read_csv(os.path.join(PREPROCESSED_FILE_PATH, '_With_regions_States_Cleaned.csv')) # read the cleaned data 

In [15]:
tx_v = data1[data1['state'] == 'TX'].copy()
ca_v = data1[data1['state'] == 'CA'].copy()
va_v = data1[data1['state'] == 'VA'].copy()
nc_v = data1[data1['state'] == 'NC'].copy()
co_v = data1[data1['state'] == 'CO'].copy()
fl_v = data1[data1['state'] == 'FL'].copy()
ma_v = data1[data1['state'] == 'MA'].copy()
md_v = data1[data1['state'] == 'MD'].copy()

In [16]:
df_list_state1 = [tx_v, ca_v, va_v, nc_v, co_v, fl_v, ma_v, md_v] #store the dataframes in a list
states = ['TX', 'CA', 'VA', 'NC', 'CO', 'FL', 'MA', 'MD'] # store the state names in a list
df_dict_state1 = dict(zip(states, df_list_state1)) #create a dictionary of dataframes with states as key

In [17]:
def preprocess_viz(df_dict):
    '''
    Cleans and preprocesses the data for future use 
    Cleaning:
        - Removes duplicate rows (if they exist)
        - Removes the state column as the dataframes are already separated 
        - Removes outliers on the basis of price column using lower limit as (Q1-1.5*IQR) and 
        upper limit as (Q3+1.5*IQR)
        
    Args:
        df_dict - the dictionary containing states as keys and the dataframe corresponding to them as values 
    Returns:
        None 
        
    '''
    
    for state, df in df_dict.items():
            
        # Cleaning the data 
        # Dropping the city column
        df.drop(['state'], axis=1, inplace=True)
        # Dropping duplicate rows (if they exist)
        df.drop_duplicates(inplace=True)
        # Removing outliers : price
        desc = df['price'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['price']>=lower_lim)&(df['price']<=upper_lim)]
        # Removing outliers : square_feet
        desc = df['square_feet'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['square_feet']>=lower_lim)&(df['square_feet']<=upper_lim)]
        
        # one hot encoding cityname
        #dummies = pd.get_dummies(df.cityname)
        #df = pd.concat([df,dummies], axis = 'columns')
        #df = df.drop('cityname',axis=1)
            
        # since the data is cleaned now, we can store it in clean_dict 
        clean_df = df.copy()
        # Saving the dataframes 
        if not os.path.exists(PREPROCESSED_FILE_PATH):
                os.makedirs(PREPROCESSED_FILE_PATH)
        df.to_csv(os.path.join(PREPROCESSED_FILE_PATH, f'{state}_viz.csv'),index=False)  

In [18]:
preprocess_viz(df_dict_state1)